In [ ]:
import pandas as pd
import numpy as np

In [ ]:
 # check current directory 


In [ ]:
pwd

In [ ]:
# read header row and remove bottom rows
imp = pd.read_csv('56-malaysias-import-sources.csv', encoding ='iso-8859-1', header = 2,skipfooter=2 )

In [ ]:
# shift tab to see the suggestion
# tab to autofill

In [ ]:
# rename columns to remove \xa0 already know it is in USD just put Import
imp.columns = ['Year', 'Month', 'Country', 'Country Code','Import'] 
# replace annoying characters with space
imp = imp.replace({'\xa0': ' '}, regex=True) 
# placing NAM in Country Code 
imp.loc[imp['Country'] == "NAMIBIA", 'Country Code'] = "NAM" 
# importing numbers
imp['Import'] = imp['Import'].replace({',': ''}, regex=True) 
imp['Import'] = imp['Import'].apply(int)

In [ ]:
# check top dataset 
imp.head()

In [ ]:
# check bottom of dataset
imp.tail()

In [ ]:
# information on dataset
imp.info()

In [ ]:
# checking if there are null values in the dataset
imp.isnull().values.any() 

In [ ]:
# obtain rows which have imports
imp[imp['Import'] != 0 ]

In [ ]:
 # number of countries/Areas where Malaysia imports
len(imp[imp['Import'] != 0 ] ["Country"].unique())

In [ ]:
# number of countries/Areas in total
len(imp['Country'].unique()) 

In [ ]:
# load export dataset
exp = pd.read_csv('55-malaysias-export-destination.csv', encoding ='iso-8859-1', header = 2,skipfooter=2 )

In [ ]:
# rename columns to remove \xa0
exp.columns = ['Year', 'Month', 'Country', 'Country Code','Export'] 
# replace annoying characters with space
exp = exp.replace({'\xa0': ' '}, regex=True) 
# placing NAM in Country Code 
exp.loc[exp['Country'] == "NAMIBIA", 'Country Code'] = "NAM" 
# exporting numbers
exp['Export'] = exp['Export'].replace({',': ''}, regex=True) 
exp['Export'] = exp['Export'].apply(int)

In [ ]:
len(exp)

In [ ]:
len(imp)

In [ ]:
exp.head()

In [ ]:
# export - import is balance of trade

In [ ]:
# drop multiple columns because you want to combine 

In [ ]:
col_exp = ['Month', 'Country', 'Export']

slim_exp = exp[col_exp]

In [ ]:
slim_exp.head()

In [ ]:
# columns that you want to keep
col_imp = ['Import']

In [ ]:
# obtain new dataset from these columns
slim_imp = imp[col_imp]


In [ ]:
pd.merge?

![](http://www.dofactory.com/Images/sql-joins.png)

In [ ]:
#combining datasets
pd.merge(slim_exp, slim_imp, how='inner', left_index=True, right_index=True)

In [ ]:
# assigning combined dataset to a variable
exp_imp = pd.merge(slim_exp, slim_imp, how='inner', left_index=True, right_index=True)#all

In [ ]:
# Balance of Trade
exp_imp['BOT'] = exp_imp['Export'] - exp_imp['Import']

In [ ]:
# checking dataset
exp_imp

In [ ]:
exp_imp.loc[exp_imp["BOT"].idxmax()]

In [ ]:
exp_imp.loc[exp_imp["BOT"].idxmin()]

In [ ]:
# filter and obtain only that of singapore
singapore = exp_imp[exp_imp['Country'] == "REPUBLIC OF SINGAPORE"]
# get months data 
Months = singapore['Month'].values
# get import export values 
sing_exp = singapore['Export']
sing_imp = singapore['Import']

In [ ]:
# plotting graphs

In [ ]:
# import matplotlib as plt for plotting graphs 

import matplotlib.pyplot as plt 

In [ ]:
# check values
sing_exp

In [ ]:
# Compare between maximum export per month and total export for singapore

# plot graphs accordingly using colours
# devide by 1e9 to make it billion

# https://matplotlib.org/api/markers_api.html 

plt.plot(Months,sing_exp/1e9, 'r*-')
plt.plot(Months, sing_imp/1e9, 'b.-')

# assign values of xtics to that of months or else it wont 
# show per date 
xticks = [Months]

plt.xticks(Months)

# plot the legend of the graph 
plt.legend(loc=5)
# label x-axis 
plt.xlabel('Month')
plt.ylabel('USD (Billion)')
plt.title('Total Import Export per Month Singapore')
plt.show()

In [ ]:
# groupby data 

In [ ]:
# this groups the data by country and sums the other columns 
group_countries = exp_imp.groupby(['Country']).sum()

In [ ]:
# view the columns in the dataset
list(group_countries)

In [ ]:
# view dataset 
group_countries

In [ ]:
# delete the month column from the dataset
del group_countries['Month']


In [ ]:
# sort by values "ascending" 
group_countries.sort_values('BOT')


In [ ]:
# view country Hong Kong Data 
group_countries[group_countries.index =='HONG KONG']


In [ ]:
######## Plot 2 graphs in one figure

In [ ]:
# sort by highest value in import column
gc_imp = group_countries.sort_values('Import', ascending=False)
# get values of top ten in the import column
m_imp = gc_imp.head(10)['Import'].values
# get the country list of the top 10
c_imp = gc_imp.head(10).index.tolist()
# get list of numbers for c_imp
y_imp = np.arange(len(c_imp))

In [ ]:
# sort by highest value in export column
gc_exp = group_countries.sort_values('Export', ascending=False)
# get values of top ten in the export column
m_exp = gc_exp.head(10)['Export'].values
# get the country list of the top 10
c_exp = gc_exp.head(10).index.tolist()
# get list of numbers for c_imp
y_exp = np.arange(len(c_exp))

In [ ]:
# Update the matplotlib configuration parameters:
import matplotlib
matplotlib.rcParams.update({'font.size': 18, 'font.family': 'serif'})
%matplotlib inline
fig = plt.figure()


In [ ]:
# plotting for sub plots 
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(20,14))

axes[0].bar(y_imp, m_imp,  width=0.5, alpha=0.5)
axes[0].set_title("Top Importers")
axes[0].set_xticks(y_imp)
axes[0].set_xticklabels(c_imp, fontsize=10)


axes[1].bar(y_exp, m_imp, align="center", width=0.5, alpha=0.5)
axes[1].set_title("Top Exporter")
axes[1].set_xticks(y_exp)
axes[1].set_xticklabels(c_exp, fontsize=10)

fig.tight_layout()

In [ ]:
# save figure as png
fig.savefig("top_imp_exp.png") 

In [ ]:
exp_imp.head()

In [ ]:
# save data into excel
group_countries.to_excel('malaysia_states.xlsx')

In [ ]:
# save data into csv
exp_imp.to_csv('imp_exp.csv',index=False)

In [ ]:
#### scatter import export 

In [ ]:
exp_imp.head()

In [ ]:
list(exp_imp['Month'])

In [ ]:
type(exp_imp['Export'][5])

In [ ]:
month = exp_imp['Month'].values

In [ ]:
export = exp_imp['Export'].values
mport = exp_imp['Import'].values 

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(20,14))

axes.scatter(month, export/1e9)
axes.scatter(month,mport/1e9,c="r")
axes.set_title("Import Export")
axes.set_xticks(month)
axes.set_xticklabels(month, fontsize=15)
axes.set_xlabel('Month',fontsize=15)
axes.set_ylabel('(USD) Billion',fontsize=15)

fig.tight_layout()

In [ ]:
# plot for BOT all at once (messy)

In [ ]:
# view list of countries by index
group_countries.index.tolist()

# obtain values  of BOT
group_countries['BOT'].values

# make list of countries and assign to variable 
countries = group_countries.index.tolist()
# obtain number values for plotting 
no_countries = np.arange(len(countries))
# obtain values for y axis
BOT_val = group_countries['BOT'].values

# change figure size 
plt.figure(figsize=(20,10))
# plot bar graph
plt.bar(no_countries, BOT_val, align='center', alpha=0.5)
# label xticks with y_pos and replace them with the object list 
plt.xticks(no_countries, countries)
# label
plt.ylabel('USD')
plt.title('BOT')
plt.xlabel('Country')

plt.show()

In [ ]:
group_countries['BOT'].head().values

In [ ]:
group_countries['BOT'].tail().values

In [ ]:
group_countries.loc[group_countries["BOT"].idxmin(5)]

In [ ]:
largest_bot = group_countries.nlargest(5, columns=['BOT'])

In [ ]:
smallest_bot = group_countries.nsmallest(5, columns=['BOT'])

In [ ]:
all_dfs = [largest_bot,smallest_bot]


In [ ]:
df = pd.concat(all_dfs)


In [ ]:
# view list of countries by index
df.index.tolist()

# obtain values  of BOT
df['BOT'].values

# make list of countries and assign to variable 
countries = df.index.tolist()
# obtain number values for plotting 
no_countries = np.arange(len(countries))
# obtain values for y axis
BOT_val = df['BOT'].values

# change figure size 
plt.figure(figsize=(20,10))
# plot bar graph
plt.bar(no_countries, BOT_val/1e9, align='center', alpha=0.5)
# label xticks with y_pos and replace them with the object list 
plt.xticks(no_countries, countries,size= 7)
# label
plt.ylabel('USD (Billion)')
plt.title('BOT')
plt.xlabel('Country')

plt.show()